In [68]:
import pandas as pd
import numpy as np
import json
import os

import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

In [89]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQD\Optimize\VaryFinalRad")
cutoffs = [2,4,8,16]
potentials = ['QHO','AHO','DW']
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
comp = 'final_tr_radius'

In [90]:
converged_only = True
best_values = []
alld = []

#for beta in betas:
for potential in potentials: 
    for cutoff in cutoffs:

        all_data = []

        for folder in folders:   

            median = []
            mean = []
            min = []
            e0 = []
            e1 = []
            e2 = []
            ratios = []

            data_path = os.path.join(folder_path, folder, potential, "{}_{}.json".format(potential, cutoff))
            
            with open(data_path, 'r') as file:
                data = json.load(file)

            num_VQD = data['num_VQD']

            converged = [all(x) for x in data['success']]
            converged_indices = [i for i, success in enumerate(converged) if success]
            converged_runs = sum(converged)

            if converged_only:
                results = [data['results'][i] for i in converged_indices]
            else:
                results = data['results']

            success_i = [i for i, success in enumerate(data['success']) if all(success)]
            successful_results = [data['results'][i] for i in success_i]
            energies =np.sort(successful_results)

            
            run_times = data['run_times']
            time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
            mean_vqe_time = np.mean(time_series)
            time_taken = np.mean(mean_vqe_time)

            for e in np.sort(energies):
                e0.append(e[0])
                e1.append(e[1])
                e2.append(e[2])

            mean_ratio = abs((np.mean(e2) - np.mean(e1)) / (np.mean(e2) - np.mean(e0)))
            median_ratio = abs((np.median(e2) - np.median(e1)) / (np.median(e2) - np.median(e0)))

            e0_exact = data['exact_eigenvalues'][0]
            e1_exact = data['exact_eigenvalues'][1]
            e2_exact = data['exact_eigenvalues'][2]
            exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

            row = {
            comp: folder,
            "potential": potential,
            "cutoff": cutoff,
            'Converged Runs': f"{converged_runs}/100",
            'Mean VQD time (s)': f"{time_taken:.2f}",
            "e0": np.median(e0),
            "e1": np.median(e1),
            "e2": np.median(e2),
            "median_ratio": median_ratio,
            "mean_ratio": mean_ratio,
            #"min_ratio": np.min(ratios),
            "e0_exact": e0_exact,
            "e1_exact": e1_exact,
            "e2_exact": e2_exact,
            "exact_ratio": exact_ratio
            }

            all_data.append(row)
            alld.append(row)

        df = pd.DataFrame(all_data)
        df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
        best = df.sort_values('diff').iloc[0][comp]

        d = {"potential": potential,
             "cutoff": cutoff,
             "best value": best}
        
        best_values.append(d)

In [91]:
best_values

[{'potential': 'QHO', 'cutoff': 2, 'best value': '1e-08'},
 {'potential': 'QHO', 'cutoff': 4, 'best value': '1e-08'},
 {'potential': 'QHO', 'cutoff': 8, 'best value': '1e-08'},
 {'potential': 'QHO', 'cutoff': 16, 'best value': '1e-08'},
 {'potential': 'AHO', 'cutoff': 2, 'best value': '1e-08'},
 {'potential': 'AHO', 'cutoff': 4, 'best value': '1e-08'},
 {'potential': 'AHO', 'cutoff': 8, 'best value': '0.01'},
 {'potential': 'AHO', 'cutoff': 16, 'best value': '0.001'},
 {'potential': 'DW', 'cutoff': 2, 'best value': '0.01'},
 {'potential': 'DW', 'cutoff': 4, 'best value': '0.0001'},
 {'potential': 'DW', 'cutoff': 8, 'best value': '1e-08'},
 {'potential': 'DW', 'cutoff': 16, 'best value': '1e-07'}]

In [92]:
ad = pd.DataFrame(alld)
ad['diff'] = ad[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
ad[(ad['potential']=='AHO') & (ad['cutoff']==16)].sort_values('diff')

,final_tr_radius,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,median_ratio,mean_ratio,e0_exact,e1_exact,e2_exact,exact_ratio,diff
57,0.001,AHO,16,26/100,186.95,0.888577,2.642083,3.259062,0.260276,0.364666,-0.001167,1.677494,1.686381,0.005266,0.255009
60,1e-05,AHO,16,3/100,210.38,0.123640,2.079044,3.298717,0.384140,0.384146,-0.001167,1.677494,1.686381,0.005266,0.378873
61,1e-06,AHO,16,1/100,215.80,0.683444,1.202838,1.666256,0.471522,0.471522,-0.001167,1.677494,1.686381,0.005266,0.466256
56,0.0001,AHO,16,6/100,210.77,0.464863,1.763573,3.013026,0.490335,0.548661,-0.001167,1.677494,1.686381,0.005266,0.485068
62,1e-07,AHO,16,3/100,211.58,12.754434,12.754472,12.754510,0.500767,0.498375,-0.001167,1.677494,1.686381,0.005266,0.495501
63,1e-08,AHO,16,2/100,214.24,16.187500,17.331878,18.569360,0.519545,0.519545,-0.001167,1.677494,1.686381,0.005266,0.514278
58,0.01,AHO,16,97/100,101.16,1.130258,2.161866,4.232629,0.667478,0.528767,-0.001167,1.677494,1.686381,0.005266,0.662211
59,0.1,AHO,16,100/100,30.70,13.167148,16.801617,24.830260,0.688379,0.594664,-0.001167,1.677494,1.686381,0.005266,0.683113


In [ ]:
df

In [ ]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df.sort_values('diff')#.iloc[0]['final_tr_radius']